# Three body term

In [10]:
# Loads stuff
import sys
sys.path.insert(0,"/home/lg3u19/OnePy")
import onetep_v0_1 as op
import pathlib
import pandas as pd
pd.set_option('display.precision',3) 
import copy

In [11]:
csv_target_path = './ProcessedData/ThreeBody'

In [22]:
ligands = ['phenol','methylphenol','catechol','fluoroaniline','hydroxyaniline']
snapshots_5 = ['24801', '32401', '17201', '9601','2001'] 
snapshots_10 = ['24801', '32401', '17201', '13401', '21001', '28601', '9601', '5801','36201','2001'] 
snapshots_25 = [2001, 3521, 5041, 6561, 8081, 9601, 11121, 12641, 14161, 15681, 17201, 18721, 20241, 21761, 23281, 24801, 26321, 27841, 29361, 30881, 32401, 33921, 35441, 36961, 38481]
snapshots_25 = [str(x) for x in snapshots_25]
snapshots_50 = [2001, 2761, 3521, 4281, 5041, 5801, 6561, 7321, 8081, 8841, 9601, 10361, 11121, 11881, 12641, 13401, 14161, 14921, 15681, 16441, 17201, 17961, 18721, 19481, 20241, 21001, 21761, 22521, 23281, 24041, 24801, 25561, 26321, 27081, 27841, 28601, 29361, 30121, 30881, 31641, 32401, 33161, 33921, 34681, 35441, 36201, 36961, 37721, 38481, 39241]
snapshots_50 = [str(x) for x in snapshots_50]

## Functions for Three Body Term

In [29]:
def load_disp_from_folder(root,exclude_list=[".","_",","],dat_file_name='disp.data'):
    """ goes through folders in root dir and loads csv files into dict by folder name
    folder correspond to ligands """
    root = pathlib.Path(root) 
    disp_dict = {}
    for folder in root.iterdir():
        if folder.is_dir():
            if folder.name[0] not in exclude_list:
                disp_dict[folder.name]=pd.read_csv(folder/dat_file_name,sep=' ')
    return disp_dict 

In [30]:
def process_disp(disp_dict,snapshots):
    """ return dictionary of mean net dispersion over a subset of snapshots"""
    mean_dict = {}
    for key in disp_dict.keys():
        # select subset of snaps
        subset = disp_dict[key].loc[[int(x) for x in snapshots],:]
        # calc net dispersion term
        subset['diff']=subset['complex']-subset['host']-subset['ligand']
        mean_dict[key]=subset.mean()['diff']
    return mean_dict

In [31]:
def series_relative_values(input_dict,snapshots,ligands,reference='phenol'):
    """ retruns series of mean values relative to reference """
    # calc relative to reference entropy
    temp_dict_rel = {}
    for ligand2 in ligands:
        if ligand2!=reference:
            temp_dict_rel[ligand2] = input_dict[ligand2] - input_dict[reference]
    return pd.Series(temp_dict_rel)

In [32]:
abc = load_disp_from_folder('./three_body',dat_file_name='disp_abc.data')

In [33]:
abc_mean = process_disp(abc,snapshots_10)
abc_mean

{'hydroxyaniline': 0.06358929999999975,
 'methylphenol': 0.054917499999999855,
 'phenol': 0.053026700000000225,
 'catechol': 0.052903100000000355,
 'fluoroaniline': 0.051022599999999675}

In [34]:
abc_rel = series_relative_values(abc_mean,snapshots_50,ligands)
abc_rel

methylphenol      1.891e-03
catechol         -1.236e-04
fluoroaniline    -2.004e-03
hydroxyaniline    1.056e-02
dtype: float64

The ABC term contributes <1% to total energies. Its change upon binding i about 0.05kcal/mol. And the change relative to phenol, so the ammount by which it would modify relative binding free energies is on the order 10^-3 kcal/mol. Well below accuracy of the term. So not needed in this system: Think about why and contrast with Grimme benchmarking study,